In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)


In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
from chart_studio.plotly import plot, iplot
import chart_studio.plotly as py

In [4]:
from plotly.graph_objs import *
import plotly.graph_objs as go

# Subject number

In [5]:
subNum = 1;
trIdx = 9;
gazeConfidenceThreshold = 0.6;

### Load dataframe
Remember to set loadParsedData, loadProcessedData.

In [6]:
dataFolderList = [];
[dataFolderList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];

dataParentFolder = "../Data/" + dataFolderList[subNum];

dataSubFolderList = [];
[dataSubFolderList.append(name) for name in os.listdir("../Data/" + dataParentFolder) if name[0] is not '.'];

dataFolder = dataParentFolder + '/' + dataSubFolderList[0] + '/' 

gazeDataFolderList = [];
[gazeDataFolderList.append(name) for name in os.listdir(dataFolder + 'PupilData') if name[0] is not '.'];

pupilSessionFolder = '/' + gazeDataFolderList[0] +  '/' 

gazeDataFolder = dataFolder + 'PupilData' + pupilSessionFolder

In [7]:
trialData = pd.read_csv( dataFolder + 'trial_results.csv')

In [8]:
trDataFiles = [i for i in trialData.columns.to_list() if '_filename' in i] 

In [9]:
logger.info('Processing subject: ' + trialData.iloc[trIdx]['ppid'] + ' t = ' + str(trialData.iloc[trIdx]['trial_num']))

INFO___main__-<module>(): - Processing subject: P_200917094202 t = 10


Rename trial data columns

In [10]:
trialData.rename(columns={"session_num":"sessionNum","trial_num":"trialNum",
                         "block_num":"blockNum","trial_num_in_block":"trialNumInBlock",
                         "start_time":"startTime","end_time":"endTime"},inplace=True)

In [11]:
# [c for c in trialData.columns[:-len(trDataFiles)]]

Get ball data

In [12]:
dataFileName = trialData.iloc[trIdx]['ball_movement_filename']
ballData = pd.read_csv( dataFolder + dataFileName)
ballData = ballData.rename(columns={"time": "frameTime"})
ballData.head()

,frameTime,ballPos_x,ballPos_y,ballPos_z,ballVel_x,ballVel_y,ballVel_z,ballRot_x,ballRot_y,ballRot_z,ballMeshRadius,ballColliderRadius
0,86.37790,-7.6228,2.1296,18.0000,7.7271,4.601,-16.7956,0.0,0.0,0.0,0.12,0.5
1,86.39147,-7.5370,2.1795,17.8134,7.7271,4.492,-16.7956,0.0,0.0,0.0,0.12,0.5
2,86.40300,-7.4511,2.2282,17.6268,7.7271,4.383,-16.7956,0.0,0.0,0.0,0.12,0.5
3,86.41599,-7.3652,2.2757,17.4402,7.7271,4.274,-16.7956,0.0,0.0,0.0,0.12,0.5
4,86.43719,-7.1935,2.3671,17.0669,7.7271,4.056,-16.7956,0.0,0.0,0.0,0.12,0.5


In [ ]:
ballData.rename(columns={"pos_x": "ballPos_x", "pos_y": "ballPos_y","pos_z": "ballPos_z"},inplace=True)
ballData.rename(columns={"rot_x": "ballRot_x", "rot_y": "ballRot_y","rot_z": "ballRot_z"},inplace=True)
ballData.head()

In [ ]:
dataFileName = trialData.iloc[trIdx]['paddle_movement_filename']
paddleData = pd.read_csv( dataFolder + dataFileName)
paddleData = paddleData.rename(columns={"time": "frameTime"})
paddleData.head()

In [ ]:
dataFileName = trialData.iloc[trIdx]['camera_movement_filename']
viewData = pd.read_csv( dataFolder + dataFileName)
viewData = viewData.rename(columns={"time": "frameTime"})
viewData.head()

In [ ]:
if( len(ballData) == 0 ):
    rawTrialData = viewData.reindex(viewData.columns.union(ballData.columns), axis=1)
else:
    rawTrialData = pd.merge(viewData, ballData, on ='frameTime',validate= 'one_to_many')

if( len(paddleData) == 0 ):
    rawTrialData = rawTrialData.reindex(viewData.columns.union(paddleData.columns), axis=1)
else:
    rawTrialData = pd.merge(rawTrialData, paddleData, on ='frameTime',validate= 'one_to_many')

In [ ]:
dataFileName = trialData.iloc[trIdx]['pupil_pupilTimeStamp_filename']
pupilTimestampData = pd.read_csv( dataFolder + dataFileName)
pupilTimestampData = pupilTimestampData.rename(columns={"time": "frameTime"})

rawTrialData = pd.merge( rawTrialData, pupilTimestampData, on ='frameTime',validate= 'one_to_many')

In [ ]:
rawTrialData['trialNumber'] = trialData.iloc[trIdx]['trialNum'].astype(int)
rawTrialData['blockNumber'] = trialData.iloc[trIdx]['blockNum'].astype(int)
rawTrialData.head()

#  Import gaze data

In [ ]:
gazeDataFolderList = []
[gazeDataFolderList.append(name) for name in os.listdir(dataFolder + 'PupilData') if name[0] is not '.']

pupilSessionFolder = '/' + gazeDataFolderList[0] 
gazeDataFolder = dataFolder + 'PupilData' + pupilSessionFolder

try:
    pupilExportsFolder = []
    [pupilExportsFolder.append(name) for name in os.listdir(gazeDataFolder + '/Exports') if name[0] is not '.']

    # Defaults to the most recent pupil export folder (highest number)
    gazePositionsDF = pd.read_csv( gazeDataFolder + '/Exports/' + pupilExportsFolder[-1] + '/gaze_positions.csv' )
    gazePositionsDF.head()
except:
    logger.exception('No gaze_positions.csv.  Process and export data in Pupil Player.')
    

In [ ]:
gazePositionsDF.sort_values(by='gaze_timestamp',inplace=True)
gazePositionsDF.head()

In [ ]:
gazePositionsDF = gazePositionsDF.rename(columns={"gaze_timestamp": "pupilTimestamp"})

# Filter by confidence

* For each trial, find the window of gaze data in gazePositionsDF, and interpolate to match those timestamps.  Then, merge, and append.

In [ ]:
gbBlTr = rawTrialData.groupby(['blockNumber','trialNumber'])
tr = gbBlTr.get_group( list(gbBlTr.groups.keys())[0])

firstTS = tr.head(1)['pupilTimestamp']
lastTS = tr.tail(1)['pupilTimestamp']
firstIdx = list(map(lambda i: i> float(firstTS), gazePositionsDF['pupilTimestamp'])).index(True) 
lastIdx = list(map(lambda i: i> float(lastTS), gazePositionsDF['pupilTimestamp'])).index(True) 
rawGazeData = gazePositionsDF.loc[firstIdx:lastIdx]


In [ ]:
filteredGazeData = rawGazeData.reset_index().drop(np.where(rawGazeData['confidence'] < gazeConfidenceThreshold )[0])

rawTrialData.head()

In [ ]:
firstTS

In [ ]:
lastTS

In [ ]:
firstIdx

In [ ]:
lastIdx

# Merge rawTrialData and trGazeData 

In [ ]:
# print(len(rawTrialData['pupilTimestamp'].unique()))
# print(len(rawTrialData['pupilTimestamp']))

In [ ]:
# print( len(rawTrialData) + len(trGazeData) )

In [ ]:
interpDF = pd.merge( rawTrialData, filteredGazeData, on ='pupilTimestamp',how='outer')
interpDF = interpDF.interpolate(method='linear',downcast='infer')
len(interpDF)

# Convert to a multilevel index

In [ ]:
interpDF.head()

In [ ]:
def convertColumn(labelIn):

    s = labelIn.split('_')
    labelOut = ''
    
    if len(s) is 2:
        
        # otherwise, its already a scalar
        (first,second) = s
        return (first,second)

    elif len(s) == 3 :
        
        (first,desc,second)= s
        return (first,second)


    else:
        return (labelIn,'')
            
def replaceGazeDataColumns(labelIn):
    
    s = labelIn.split('_')
    
    if( labelIn.split('_')[-1] == 'x' or  labelIn.split('_')[-1] == 'y' or labelIn.split('_')[-1] == 'z' ):
        
        top = '-'.join(labelIn.split('_')[:-1])
        bottom = labelIn.split('_')[-1]
        return (top,bottom)
    
    else:
        
        return (labelIn,'')


Convert interpolated dataframe and gaze dataframe to multiinedex

In [ ]:
newColList = [convertColumn(c) for c in interpDF.columns[:len(rawTrialData.columns)]]
newGazeColList = [ replaceGazeDataColumns(c) for c in interpDF.columns[(len(rawTrialData.columns)):] ]
newColList.extend(newGazeColList)
interpDF.columns = pd.MultiIndex.from_tuples(newColList)
interpDF.head()

Assign trial data to interception dataframe or calibration trial dataframe

In [ ]:
rawDataDF = False
calibDF = False

if (trialData.iloc[trIdx]['trialType'] == 'interception'):
    if( rawDataDF is False): 
        rawDataDF = interpDF
    else:
        rawDataDF = rawDataDF.append(interpDF)
    
elif(trialData.iloc[trIdx]['trialType'] == 'CalibrationAssessment'):

    if( calibDF is False): 
        calibDF = interpDF
    else:
        calibDF = rawDataDF.append(interpDF)
    

Convert trial data to multiindex

In [ ]:
# calibDF.keys()

# Convert trialData to multiindex

In [ ]:
newColList = [convertColumn(c) for c in trialData.columns[:-len(trDataFiles)]]
newColList.extend([replaceGazeDataColumns(c) for c in trialData.columns[-len(trDataFiles):]])
trialData.columns = pd.MultiIndex.from_tuples(newColList)
trialData.head()

# Experiment info

# Plot some values to compare against interpolation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.ion()
fig, ax = plt.subplots()  # Create a figure containing a single axes.


ax.plot(rawTrialData['frameTime'], rawTrialData['paddlePos_x'],linewidth=2)
ax.plot(interpDF['frameTime'], interpDF[('paddlePos','x')],'-.',linewidth=1)


# Explore data

In [ ]:
interpDF.keys()

In [ ]:
interpDF['cameraPos'].head() - interpDF[('gaze-point-3d')].head()

In [ ]:
def calcGIW(sessionDictIn):

    def calcGIW(rowIn):
        
        a =  rowIn['gaze-point-3d'] - rowIn['cameraPos']
        cycGIWDir_XYZ = a / np.linalg.norm(a)
        return {('cycGIWDir','x'): cycGIWDir_XYZ[0],('cycGIWDir','y'): cycGIWDir_XYZ[1],('cycGIWDir','z'): cycGIWDir_XYZ[2]}

    cycGIWDf = sessionDictIn['processedExp'].apply(lambda rowIn: calcGIW(rowIn),axis=1)
    cycGIWDf = pd.DataFrame.from_records(cycGIWDf)

    sessionDictIn['processed'] = sessionDictIn['processed'].join(cycGIWDf)
    logger.info('Added sessionDict[\'processedExp\'][\'cycGIWDir\']')

    return sessionDictIn

In [ ]:

interpDF.apply(lambda rowIn: calcGIW(rowIn),axis=1)
